# ERC-4626: vault ecosystem comparison across chains

This notebook serves both as a coding tutorial and a useful data analytics tool for ERC-4626 vaults. 

- In this notebook, we examine different ERC-4626 vaults across different EVM blockchains   
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- Read general instructions [how to run the tutorials](./)
- See `ERC-4626 scanning all vaults onchain` example in tutorials first how to build a vault database as local `vault_db.pickle` file.




## Setup

- Set up notebook renderinb parmaeters

In [1]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [2]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 7828 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [3]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["ERC-7575"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7575())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Mgmt fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Perf fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")

df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

source_df = df

display(df.sort_values("Total events", ascending=False).head())



DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Denomination, NAV, Protocol, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


Symbol           Name  \
Chain    Address                                                               
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a   UB-WETH        UB-WETH   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701      sDAI   Savings xDAI   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  yvUSDC-A  USDC yVault-A   
         0xbb287e6017d3deb0e2e65061e8684eab21060123  yvUSDT-A  USDT yVault-A   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952  bathUSDC   BathToken v1   

                                                    Denomination  \
Chain    Address                                                   
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a         WETH   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701        WXDAI   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0         USDC   
         0xbb287e6017d3deb0e2e65061e8684eab21060123         USDT   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952         USDC   

                                                              NAV  \
Chain    Address                                                    
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a        383.58   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701 74,774,092.80   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0    570,858.69   
         0xbb287e6017d3deb0e2e65061e8684eab21060123    462,288.76   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952     60,666.28   

                                                               Protocol  \
Chain    Address                                                          
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a  <unknown ERC-4626>   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701  <unknown ERC-4626>   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0            Yearn v3   
         0xbb287e6017d3deb0e2e65061e8684eab21060123            Yearn v3   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952  <unknown ERC-4626>   

                                                      Mgmt fee   Perf fee  \
Chain    Address                                                            
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a  <unknown>  <unknown>   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701  <unknown>  <unknown>   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  <unknown>  <unknown>   
         0xbb287e6017d3deb0e2e65061e8684eab21060123  <unknown>  <unknown>   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952  <unknown>  <unknown>   

                                                                          Shares  \
Chain    Address                                                                   
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a       382.092618511635223502   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701  63831623.699534889120512711   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0                 484583.69304   
         0xbb287e6017d3deb0e2e65061e8684eab21060123                390039.532892   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952              2.7272963112E-8   

                                                             First seen  \
Chain    Address                                                          
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a 2025-01-13 06:53:05   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701 2023-09-28 12:13:30   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0 2023-10-26 16:45:48   
         0xbb287e6017d3deb0e2e65061e8684eab21060123 2023-12-07 17:02:20   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952 2022-05-24 23:22:04   

                                                                                       _detection_data  \
Chain    Address                                                                                         
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a  ERC4262VaultDetection(chain=1868, address='0x2...   
Gnosis   0xaf204776c7245bf4147c261

## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [4]:
nav_threshold = 100_000
broken_max_nav_threshold = 999_000_000_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]

# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby(level='Chain').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby(level='Chain')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby(level='Chain').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby(level='Chain').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby(level='Chain').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby(level='Chain').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby(level='Chain').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby(level='Chain').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,,
Arbitrum,1916,"122,849,959.50",342.00,660,56.00,18.00,26.00,219.00
Avalanche,280,"61,745,687.18",39.00,82,6.00,1.00,1.00,48.00
Base,1158,"291,648,337.16",581.00,423,49.00,26.00,12.00,94.00
Berachain,233,"321,753,866.85",29.00,17,11.00,0.00,0.00,2.00
Binance,316,"220,607,730.24",18.00,84,9.00,0.00,0.00,25.00
Blast,106,"2,072,327.75",2.00,14,2.00,0.00,0.00,32.00
Celo,5,"3,189,899.40",0.00,2,1.00,0.00,0.00,0.00
Ethereum,2123,"9,457,790,709.66",527.00,707,179.00,16.00,14.00,177.00
Gnosis,85,"77,409,617.42",12.00,35,5.00,0.00,0.00,0.00


## Vaults per protocol summary

- Break down by identified protocol
- Because of the brokeness of EVM, Solidity and smart contract development practices, protocol-rules are hand-maintained heurestics and there can false positives and negatives

In [5]:
df = source_df.copy()
df = df.reset_index()
# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]
# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby('Protocol').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby('Protocol')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby('Protocol').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby('Protocol').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby('Protocol').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby('Protocol').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby('Protocol').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby('Protocol').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
summary_df = summary_df.sort_values("Total vault TVL/NAV in stable vaults", ascending=False)
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Protocol,,,,,,,,
<unknown ERC-4626>,5597,"8,425,016,600.96",0.00,"2,004.00",208.00,0.00,12.00,0.00
Morpho,184,"1,397,487,651.92",184.00,94.00,56.00,0.00,0.00,184.00
Fluid,57,"491,407,492.49",57.00,25.00,7.00,0.00,0.00,0.00
Euler Vault Kit,267,"164,962,943.06",267.00,68.00,20.00,0.00,0.00,0.00
Yearn v3,128,"68,537,999.51",128.00,65.00,18.00,0.00,0.00,0.00
Superform,129,"31,587,803.88",129.00,51.00,10.00,13.00,0.00,0.00
Kiln Metavault,71,"18,682,487.87",71.00,63.00,8.00,0.00,0.00,0.00
Yearn compounder,71,"16,684,963.12",71.00,12.00,1.00,0.00,0.00,0.00
Term Finance,16,"11,328,468.36",16.00,15.00,5.00,0.00,0.00,0.00


## Vault deployment history

- Show how much history we have for each chain


In [6]:
# Assuming your DataFrame is named 'df'
df = source_df
seen_df = df.groupby(level='Chain')['First seen'].agg(['min', 'max']).reset_index()

# Rename columns for clarity
seen_df.columns = ['Chain', 'First vault deployed', 'Last vault deployed']

seen_df = seen_df.set_index("Chain")

display(seen_df)

,First vault deployed,Last vault deployed
Chain,,
Arbitrum,2022-03-28 09:03:11,2025-04-03 20:38:18
Avalanche,2022-04-11 00:05:24,2025-04-10 00:26:55
Base,2023-08-04 15:36:07,2025-04-09 14:47:09
Berachain,2025-01-26 00:06:03,2025-04-09 07:50:02
Binance,2022-05-27 17:25:18,2025-04-08 11:29:07
Blast,2024-02-29 20:15:37,2024-12-03 15:38:15
Celo,2022-08-08 21:14:10,2025-01-29 23:13:18
Ethereum,2019-06-11 06:17:19,2025-04-09 17:13:47
Gnosis,2022-05-01 18:47:35,2025-04-07 08:35:55


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [7]:
largest_threshold = 20
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100] 
largest_df = largest_df[largest_df["Stablecoin denominated"] == True] 
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")


display(largest_df.head(largest_threshold))


,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
Name,,,,,,,
Savings USDS,"2,979,613,394.52",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,28155,<unknown>,<unknown>
Staked USDe,"2,162,640,474.16",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,69230,<unknown>,<unknown>
Ethereal Pre-deposit Vault,"892,758,591.78",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,49121,<unknown>,<unknown>
Savings Dai,"518,495,409.71",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,72590,<unknown>,<unknown>
Bridged USDC (Stargate)Vault,"307,578,099.32",Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,135337,<unknown>,<unknown>
Usual Boosted USDC,"246,334,839.20",Ethereum,0xd63070114470f685b75b74d60eec7c1113d33a3d,USDC,17461,0.00,0.00
Fluid USD Coin,"210,501,860.29",Ethereum,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,USDC,17564,<unknown>,<unknown>
Fluid Tether USD,"200,522,647.05",Ethereum,0x5c20b550819128074fd538edf79791733ccedd18,USDT,10433,<unknown>,<unknown>
Staked USDX,"199,665,946.99",Binance,0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5449,<unknown>,<unknown>


## Largest USD vault per chain

- Get the largest vault of each chain

In [8]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby('Chain')['NAV'].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

,,NAV,Address,Denomination,Total events,Mgmt fee,Perf fee
Chain,Name,,,,,,
Arbitrum,Fluid USD Coin,"28,464,410.55",0x1a996cb54bb95462040408c06122d45d6cdb6096,USDC,33675,<unknown>,<unknown>
Avalanche,EVK Vault eUSDC-2,"21,664,340.50",0x39de0f00189306062d79edec6dca5bb6bfd108f9,USDC,349,<unknown>,<unknown>
Base,Seamless USDC Vault,"50,257,791.25",0x616a4e1db48e22028f6bbf20444cd3b8e3273738,USDC,79414,0.00,0.00
Berachain,Bridged USDC (Stargate)Vault,"307,578,099.32",0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,135337,<unknown>,<unknown>
Binance,Staked USDX,"199,665,946.99",0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5449,<unknown>,<unknown>
Blast,Wasabi USDB Vault,"1,569,206.51",0x4bed2a922654cacc2be974689619768fabf24855,USDB,92762,<unknown>,<unknown>
Celo,Staked EURA,"3,189,898.33",0x004626a008b1acdc4c74ab51644093b155e59a23,EURA,223,<unknown>,<unknown>
Ethereum,Savings USDS,"2,979,613,394.52",0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,28155,<unknown>,<unknown>
Gnosis,Savings xDAI,"74,774,092.80",0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,532670,<unknown>,<unknown>


## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [9]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
UB-WETH,717677,Soneium,0x232554b4b291a446b4829300bec133fbb07a8f2a,WETH,383.58,87 days 14:34:27.109282,529392,188285
Savings xDAI,532670,Gnosis,0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"74,774,092.80",560 days 09:14:02.109282,287498,245172
USDC yVault-A,490172,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"570,858.69",532 days 04:41:44.109282,465206,24966
USDT yVault-A,458210,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"462,288.76",490 days 04:25:12.109282,407692,50518
BathToken v1,297308,Optimism,0xe0e112e8f33d3f437d1f895cbb1a456836125952,USDC,"60,666.28",1051 days 22:05:28.109282,150209,147099
Wasabi WETH Vault,242858,Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,"1,007.37",406 days 01:11:55.109282,161151,81707
UB-USDC,225476,Soneium,0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41,USDC.e,"1,234,083.69",87 days 14:35:21.109282,160907,64569
UB-ASTR,206178,Soneium,0x85a4fb48c7f9383083864d62abeccdf318fd8e6f,ASTR,"38,040,007.32",87 days 14:34:51.109282,103483,102695
pufETH,167878,Ethereum,0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"69,830.82",435 days 01:42:57.109282,151577,16301


## Most historically active vault per chain

- Vaults with most deposit and redeem events

In [10]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby('Chain')['Total events'].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"121,775.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"967,132.98",455 days 12:55:07.109282,119754,2021
Avalanche,GoGoPool Liquid Staking Token,"67,268.00",0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,WAVAX,"860,520.18",736 days 00:16:47.109282,41541,25727
Base,Moonwell Flagship ETH,"140,868.00",0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"13,396.42",302 days 08:30:17.109282,85541,55327
Berachain,Bridged USDC (Stargate)Vault,"135,337.00",0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,"307,578,099.32",74 days 21:21:29.109282,80977,54360
Binance,kUSDT,"42,810.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"2,593,079.58",668 days 14:18:49.109282,25334,17476
Blast,Wasabi WETH Vault,"242,858.00",0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,"1,007.37",406 days 01:11:55.109282,161151,81707
Celo,Wrapped mcUSD,241.00,0x5f71fe197fd9aad95b7b5510a98882769622112a,mcUSD,"32,034.68",938 days 01:53:35.109282,180,61
Ethereum,pufETH,"167,878.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"69,830.82",435 days 01:42:57.109282,151577,16301
Gnosis,Savings xDAI,"532,670.00",0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"74,774,092.80",560 days 09:14:02.109282,287498,245172


## Oldest vaults

- Show oldest vaults

In [11]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby('Chain')['Age'].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1049 days 06:12:26.109282,WETH,18.00,"2,335.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1093 days 04:30:10.109282,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,588 days 05:33:39.109282,WETH,0.97,"1,117.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,74 days 21:21:29.109282,USDC.e,"307,578,099.32","135,337.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1018 days 00:34:53.109282,NRCH,"1,188,908.73","1,195.00"
Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,Wasabi WETH Vault,406 days 01:11:55.109282,WETH,"1,007.37","242,858.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1081 days 10:01:55.109282,THOR,"79,580,695.56","19,579.00"
Gnosis,0x004626a008b1acdc4c74ab51644093b155e59a23,Staked EURA,587 days 07:45:37.109282,EURA,"510,477.11","3,585.00"
Mantle,0x8529019503c5bd707d8eb98c5c87bf5237f89135,Karak - mETH,352 days 03:15:26.109282,mETH,488.97,"13,028.00"


## All vaults

- List of all stablecoin vaults

In [12]:
all_df = df.reset_index()

min_nav = 10_000

stablecoin_denominated = all_df["Stablecoin denominated"] == True
all_df = all_df[stablecoin_denominated]
all_df = all_df[all_df["NAV"] < broken_max_nav_threshold]
all_df = all_df[all_df["NAV"] >= min_nav]
all_df = all_df[["NAV", "Chain", "Protocol", "Address", "Name", "Age", "Denomination", "Total events"]]
all_df = all_df.set_index("Name").sort_values("NAV", ascending=False)

print(f"Displaying {len(all_df)} stablecoin vaults w/min NAV of {min_nav} USD")
pd.options.display.max_rows = None
display(all_df)


Displaying 544 stablecoin vaults w/min NAV of 10000 USD


,NAV,Chain,Protocol,Address,Age,Denomination,Total events
Name,,,,,,,
Savings USDS,"2,979,613,394.52",Ethereum,<unknown ERC-4626>,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,205 days 07:50:45.109282,USDS,28155
Staked USDe,"2,162,640,474.16",Ethereum,<unknown ERC-4626>,0x9d39a5de30e57443bff2a8307a4256c8797a3497,506 days 13:21:21.109282,USDe,69230
Ethereal Pre-deposit Vault,"892,758,591.78",Ethereum,<unknown ERC-4626>,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,56 days 06:13:21.109282,USDe,49121
Savings Dai,"518,495,409.71",Ethereum,<unknown ERC-4626>,0x83f20f44975d03b1b09e64809b757c47f942beea,743 days 11:02:45.109282,DAI,72590
Spark DAI Vault,"325,124,600.68",Ethereum,Morpho,0x73e65dbd630f90604062f6e02fab9138e713edd9,377 days 07:07:45.109282,DAI,96
Bridged USDC (Stargate)Vault,"307,578,099.32",Berachain,<unknown ERC-4626>,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,74 days 21:21:29.109282,USDC.e,135337
Usual Boosted USDC,"246,334,839.20",Ethereum,Morpho,0xd63070114470f685b75b74d60eec7c1113d33a3d,259 days 02:33:21.109282,USDC,17461
Fluid USD Coin,"210,501,860.29",Ethereum,Fluid,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,416 days 20:12:09.109282,USDC,17564
Fluid Tether USD,"200,522,647.05",Ethereum,Fluid,0x5c20b550819128074fd538edf79791733ccedd18,416 days 20:12:09.109282,USDT,10433
